In [0]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
data = pd.read_csv("race.csv",encoding='utf-8')
data = data.dropna(subset=['record'])
# data = data.dropna() # 이거로 하면 cost 5자리수 나옴


y = data['record']
X = data.loc[:, ["lane","sex","age","jockey_w","jockey","dandivi","yeondivi","cure_in_1m",
                 "weight_diff","raw_weight","weight_added","prev1_velo","horse_level_num"]]

#print(X.shape)

# sex는 dummy 함수 처리
sex_mod = pd.get_dummies(X['sex'])
X = X.join(sex_mod)
X = X.drop(columns=['sex'])
#print(X.columns)
X = X.as_matrix()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
train_size = len(X_train)
print("train_size : ",train_size)

size_x = len(X_train[0])
print("size_x : ",size_x)

# 설정하는 부분
learning_rate = 0.001
training_epochs = 100
batch_size = 16


train_size :  48596
size_x :  15


In [8]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, size_x])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)



W1 = tf.get_variable(shape=[size_x, 16], name='weight1', initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([16]))
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)


W2 = tf.get_variable(shape=[16, 16], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([16]))
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)


W3 = tf.get_variable(shape=[16, 1], name='weight3', initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(layer2, W3) + b3

cost = tf.reduce_mean(tf.square(hypothesis - Y))
#cost = tf.losses.mean_squared_error(labels=Y, predictions=hypothesis)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_size / batch_size)

    for i in range(total_batch):
        batch_xs = X_train[i*16:(i+1)*16]
        batch_ys = y_train[i*16:(i+1)*16].reshape(16,1)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, train], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Epoch: 0001 cost = 28094.859344171
Epoch: 0002 cost = 1490.603507305
Epoch: 0003 cost = 1160.870104195
Epoch: 0004 cost = 1112.735124510
Epoch: 0005 cost = 1048.184310119
Epoch: 0006 cost = 965.798328665
Epoch: 0007 cost = 899.348867759
Epoch: 0008 cost = 863.745838355
Epoch: 0009 cost = 841.462584982
Epoch: 0010 cost = 820.512091593
Epoch: 0011 cost = 783.939630382
Epoch: 0012 cost = 753.501359575
Epoch: 0013 cost = 725.837088242
Epoch: 0014 cost = 711.526376578
Epoch: 0015 cost = 681.193405388
Epoch: 0016 cost = 660.277234467
Epoch: 0017 cost = 640.147135057
Epoch: 0018 cost = 620.714924890
Epoch: 0019 cost = 598.821959047
Epoch: 0020 cost = 581.655060392
Epoch: 0021 cost = 568.612276752
Epoch: 0022 cost = 553.008758100
Epoch: 0023 cost = 532.610906463
Epoch: 0024 cost = 520.938743805
Epoch: 0025 cost = 509.705150261
Epoch: 0026 cost = 496.308639574
Epoch: 0027 cost = 479.701210846
Epoch: 0028 cost = 473.791683990
Epoch: 0029 cost = 463.873432020
Epoch: 0030 cost = 450.863574380
Epoc

In [15]:
y_test = y_test.reshape(-1,1)

acc = tf.equal(tf.argmax(hypothesis, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(acc, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: X_test, Y: y_test, keep_prob:1}))

Accuracy: 1.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


(20828,)